# Test run for segmentation with + and - samples

In [3]:
# import newly developed scripts
import sys
sys.path.append('/Users/nweir/Dropbox/code/csth-imaging')
from csth_analysis import czi_io, find_cells, segment_cells, foci
import numpy as np
import os
from skimage import io, measure
from scipy.ndimage import filters
import matplotlib.pyplot as plt
from scipy import stats
import pickle
from skimage.feature import canny
from pyto_segmenter import PexSegment
from scipy.ndimage.morphology import distance_transform_edt
from skimage.morphology import watershed
import imp
import csth_analysis
%matplotlib inline

In [4]:
os.chdir('/Users/nweir/Dropbox/chris_imaging/imaging_test/th_sample_czi/')
positive_finder = find_cells.MultiFinder('/Users/nweir/Dropbox/chris_imaging/imaging_test/th_sample_czi/LC3-p62_HEK_dTMEM_No Torin_10Pos_1_AiryscanProcessing.czi',
                                    bg_filename = '/Users/nweir/Dropbox/chris_imaging/imaging_test/th_sample_czi/LC3-p62_HEK_dVPS37A_No Torin_EMPTY_AiryscanProcessing.czi')

In [5]:
positive_splitter = segment_cells.CellSplitter(positive_finder)

In [7]:
positive_splitter.segment_nuclei()

input image has 4 dimensions (multiple imgs)
segmenting image #1 of 10...
raw image imported.
performing gaussian filtering...
Image smoothed.
preprocessing complete.
thresholding...
mode = threshold.
thresholding complete.
filling holes in objects.
generating distance map...
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
passing segmented objs and seeds to Nuclei instance...

segmenting image #2 of 10...
raw image imported.
performing gaussian filtering...
Image smoothed.
preprocessing complete.
thresholding...
mode = threshold.
thresholding complete.
filling holes in objects.
generating distance map...
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
passing segmented objs and seeds to Nuclei instance...

segmenting image #3

In [12]:
os.system('say "Done"')

0

In [8]:
positive_splitter.segment_cells(488)
os.system('say "Done"')

checking that nuclei have already been segmented...
generating cell masks...
log-transforming arrays...
applying gaussian filter...
computing p-value transformation...
 computing p-val xform for slice 1 out of 10
 computing p-val xform for slice 2 out of 10
 computing p-val xform for slice 3 out of 10
 computing p-val xform for slice 4 out of 10
 computing p-val xform for slice 5 out of 10
 computing p-val xform for slice 6 out of 10
 computing p-val xform for slice 7 out of 10
 computing p-val xform for slice 8 out of 10
 computing p-val xform for slice 9 out of 10
 computing p-val xform for slice 10 out of 10
converting to binary...

generating cell masks...

generating mask #1
labeling contiguous objects...
eliminating objects w/volume < 100,000 px...
pruning labels...
appending outputs...
mask #1 complete.

generating mask #2
labeling contiguous objects...
eliminating objects w/volume < 100,000 px...
pruning labels...
appending outputs...
mask #2 complete.

generating mask #3
label

0

In [15]:
positive_splitter.filenames

AttributeError: 'CellSplitter' object has no attribute 'filenames'

In [9]:
## write
os.chdir('/Users/nweir/Dropbox/chris_imaging/imaging_test/th_sample_czi/')
file_path = "positive_splitter.pickle"
max_bytes = 2**31 - 1
 
bytes_out = pickle.dumps(positive_splitter)
with open(file_path, 'wb') as f_out:
    for idx in range(0, len(bytes_out), max_bytes):
        f_out.write(bytes_out[idx:idx+max_bytes])


In [ ]:
## write
os.chdir('/Users/nweir/Dropbox/chris_imaging/imaging_test/th_sample_czi/')
file_path = "negative_splitter.pickle"
max_bytes = 2**31 - 1
 
bytes_out = pickle.dumps(negative_splitter)
with open(file_path, 'wb') as f_out:
    for idx in range(0, len(bytes_out), max_bytes):
        f_out.write(bytes_out[idx:idx+max_bytes])


In [ ]:
## read
bytes_in = bytearray(0)
input_size = os.path.getsize(file_path)
with open(file_path, 'rb') as f_in:
    for _ in range(0, input_size, max_bytes):
        bytes_in += f_in.read(max_bytes)
data2 = pickle.loads(bytes_in)

assert(data == data2)